In [ ]:
from Generator import *
import json

In [ ]:
#create Generator dictionary
gen = g.dict()

In [ ]:
class UnitTest:
    
    gen:dict = None  # Generator dictionary
    xls:dict = None  # Excel dictionary
    eps:float = 0.05 # max rel accuracy for test to pass
    p_sector_gen = None
    
    # read excel-dictionary
    def set_excel(self, excel_path):
        with open(excel_path, 'r') as fp:
            self.xls = json.load(fp)
        fp.close()
        
    # set Generator dictionary
    def set_dict(self, gen):
        self.gen = gen
    
    def check(self, sector:str, filter: int):
        # compare data
        # parameters: 
        #             sector: Sector to be checked (i.e. 'b18') 
        #             filter: printout filter
        #             == 1 non existing row and column values
        #             == 2 values not determined (None)
        #             == 4 values that differ by more than 1%
        #             == 8 values correct
        #             or combinations (i.e. 2+4 = 6)
        gen = self.gen
        xls = self.xls       
        eps = self.eps
        
        #counter
        vne = vn = vnc = vcc = vnz = 0
        
        if gen == None or xls == None:
            print('dictionaries to compare missing')
            return
        
        sector_xls = xls[sector]
        sector_gen = gen[sector]
        self.p_sector_gen = sector_gen

        #rows
        for row in sector_xls:
            if len(row) < 1:
                continue

            if row not in sector_gen:
                vne += 1
                if filter & 1 != 0:
                    print('row "' + row + '" does not exist\n')
                continue

            #columns
            for col in sector_xls[row]:
                if col not in sector_gen[row]:
                    vne += 1
                    if filter & 1 != 0:
                        print('column "' + col + '" does not exist (' + row + ')')
                    continue

                # exclude string values
                if type(sector_xls[row][col]).__name__ != 'str':
                    # print None - values
                    if sector_gen[row][col] == None:
                        vn += 1
                        if filter & 2 != 0:
                            print('%-50s: %15.2f is None' % (row + '.' + col, sector_xls[row][col]))


                    # treat 0 or non 0 values different
                    # print differences
                    elif sector_xls[row][col] != 0:
                        #relative error
                        rerr = abs((sector_gen[row][col] - sector_xls[row][col]) / sector_xls[row][col])
                        if rerr > eps:
                            vnc += 1
                            if filter & 4 != 0:
                                print('%-50s: %15.2f %15.2f Excel' % (row + '.' + col, sector_gen[row][col], sector_xls[row][col]))
                        else:
                            vcc += 1
                            if filter & 8 != 0:
                                print('%-50s: %15.2f %15.2f Correct' % (row + '.' + col, sector_gen[row][col], sector_xls[row][col]))
                    # print 
                    else:
                        if sector_gen[row][col] != 0:
                            vnz += 1
                            if filter & 16 != 0:
                                print('%-50s: %15.2f %15.2f Excel' % (row + '.' + col, sector_gen[row][col], sector_xls[row][col]))
        
        # Statistics
        print('columns or rows not found: ' + str(vne))
        print('values not calculated: ' + str(vn))
        print('values wrong: ' + str(vnc))
        print('values not in Excel: ' + str(vnz))
        print('values correct: ' + str(vcc))
        print('sum: ' + str(vne + vn + vnc + vnz + vcc))
              

In [ ]:
ut = UnitTest()
ut.set_excel('./Excel_values.json')
ut.set_dict(gen)
ut.check('i18', 4)